In [130]:
import numpy as np
import pandas as pd
import re#Regular exp,used to find text in the doc.
!pip install nltk
from nltk.corpus import stopwords#Natual lang tool kit,stopwords are removed that does't impact our model perf.
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer#text to feature vector.
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [131]:
import nltk
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [132]:
#Data Preprocessing
news_dset = pd.read_csv(r'D:\job\DATA SCIENTIST\Job\ML PROJECTS\dup\train.csv')

In [133]:
print(news_dset)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [134]:
news_dset.shape

(20800, 5)

In [135]:
news_dset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [136]:
#Checking for null vals
news_dset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [166]:
#Filling with empty str''
news_dset = news_dset.fillna('')

In [167]:
news_dset.isnull().sum()

id         0
title      0
author     0
text       0
label      0
combine    0
dtype: int64

In [168]:
#Feat Eng
news_dset['combine'] = news_dset['author']+' '+news_dset['title']

In [169]:
news_dset.head()

,id,title,author,text,label,combine
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [170]:
print(news_dset['combine'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: combine, Length: 20800, dtype: object


In [171]:
#Separating data and label 
X = news_dset.drop(columns = 'label',axis = 1)
Y = news_dset['label']


In [172]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [173]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


In [218]:
#Stemming
port_stem = PorterStemmer()

In [222]:
def stemming(combine):
    stemmed_content = re.sub('[^a-zA-Z)]',' ', combine)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [242]:
news_dset['combine'] = news_dset['combine'].apply(stemming)


In [243]:
print(news_dset['combine'])#After Processing

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: combine, Length: 20800, dtype: object


In [244]:
X = news_dset['combine'].values
Y = news_dset['label'].values

In [245]:
print(X)

['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv']


In [246]:
print(Y)

[1 0 1 ... 0 1 1]


In [247]:
print(X.shape,Y.shape)

(20800,) (20800,)


In [248]:
#Tfidvectorizer:Converts textual data to numerical.
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [249]:
print(X)

  (0, 15611)	0.29086078121993464
  (0, 13401)	0.2561060982875654
  (0, 8859)	0.36291114581949546
  (0, 8581)	0.29157460096692783
  (0, 7644)	0.24738509222740032
  (0, 6957)	0.21832944965670537
  (0, 4935)	0.2328772570642259
  (0, 3773)	0.27002340032597333
  (0, 3579)	0.3592156616836022
  (0, 2943)	0.24637980801289425
  (0, 2473)	0.3669590906933938
  (0, 267)	0.2695922161637983
  (1, 16720)	0.3025133324406114
  (1, 6769)	0.1915634698880887
  (1, 5462)	0.7185958931261553
  (1, 3547)	0.26531272184639704
  (1, 2800)	0.1920860630210844
  (1, 2214)	0.3691535659080562
  (1, 1884)	0.1561467100400882
  (1, 1488)	0.29574485088142083
  (2, 15536)	0.41544962664721613
  (2, 9572)	0.49351492943649944
  (2, 5926)	0.3474613386728292
  (2, 5348)	0.3866530551182615
  (2, 3083)	0.46097489583229645
  :	:
  (20797, 13051)	0.2483705036831893
  (20797, 12280)	0.27276402145717243
  (20797, 12073)	0.24790022252744132
  (20797, 10258)	0.0804189541935242
  (20797, 9540)	0.17463635692029988
  (20797, 9471)	0.2939

In [250]:
#Splitting the dataset to train and test split
X_train,x_test,Y_train,y_test = train_test_split(X,Y,test_size = 0.2,stratify=Y,random_state=2)
#labels of X_train stored in x_test.

In [251]:
#Training the model:Logistic Regression
model = LogisticRegression()

In [252]:
model.fit(X_train,Y_train)

LogisticRegression()

Evalution

In [253]:
#Accuracy score on the training data
X_train_Predict = model.predict(X_train)#model predictions are stored in X_train_Predict var.
Training_data_accuracy = accuracy_score(X_train_Predict,Y_train)#Mesuring the acc on Model Training labels


In [254]:
print("Accuracy score of training data:",Training_data_accuracy)

Accuracy score of training data: 0.98671875


In [255]:
#Accuracy score on the test data
X_test_Predict = model.predict(x_test)
Testing_data_accuracy = accuracy_score(X_test_Predict,y_test)

In [256]:
print("Accuracy score of test data:",Testing_data_accuracy)

Accuracy score of test data: 0.979326923076923


In [265]:
#Making predictive sys:News is real or fake.
X_sample = x_test[3]
prediction = model.predict(X_sample)

In [266]:
if prediction[0]==0:
    print('News is Real')
else:
    print("News is fake")

News is Real


In [268]:
#Testing the dataset
print(y_test[3])

0


In [269]:
#0-->fake
#1-->Real